In [1]:
import numpy as np
import pandas as pd
from dLoader import DataLoader

In [2]:
# Binance API REST Call without api keys
def millisecond(date, place=10 ** 6):
    return pd.Timestamp(date).value // place

def url(symbol, interval, start=None, end=None, limit=1000):
    # Look back data of 3 years
    # interval: m, d, M
    # Max limit: 1000
    base_url = "https://api.binance.us/api/v3/klines?"
    symbol = "symbol=" + symbol
    interval = "&interval=" + interval
    start = "&startTime=" + str(millisecond(start)) if start is not None else ""
    end = "&endTime=" + str(millisecond(end)) if end is not None else ""
    limit = "&limit=" + str(limit)
    return base_url + symbol + interval + start + end + limit

def get_data(symbol, interval, start=None, end=None, limit=1000):
    raw = pd.read_json(url(symbol, interval, start=start, end=end, limit=limit))
    # Making change to DataFrame
    date = pd.to_datetime(raw.iloc[:, 0], unit='ms')
    date.name = None
    columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df = raw.iloc[:, 1:6]
    df.columns = columns
    df.index = date
    return df

In [136]:
def query_yahoo_finance(symbol, start=None, end=None, interval=None):
    # Max 60 days look back, 300 rows per query
    valid_interval = ['1m', '2m', '5m', '15m', '30m', '60m', '90m', 
                      '1h', '1d', '5d', '1wk', '1mo', '3mo']
    assert interval in valid_interval, "Valid intervals: {}".format(valid_interval)
    # Base Yahoo Finance Query API URL
    query_url = "https://query1.finance.yahoo.com/v8/finance/chart/"
    interval = "&interval=" + interval if interval is not None else ""
    period1 = "&period1=" + str(millisecond(start, place=10**9)) if start is not None else ""
    period2 = "&period2=" + str(millisecond(end, place=10**9)) if end is not None else ""
    url = f"{query_url}{symbol}?symbol={symbol}{period1}{period2}{interval}"
    # Get meta data
    meta = pd.read_json(url).loc['result', 'chart'][0]   
    # Change to DataFrame using meta data
    timestamp = pd.to_datetime(meta['timestamp'], unit='s')
    columns = ['open', 'high', 'low', 'close', 'volume']
    df = pd.DataFrame.from_dict(meta['indicators']['quote'][0]).reindex(columns=columns)
    df.columns = [col.capitalize() for col in columns]
    df.index = timestamp.tz_localize('UTC').tz_convert('US/Eastern')
    return df

In [3]:
# Load qyld csv
pd.read_csv('qyld_full-holdings_20220228.csv')

,% of Net Assets,Ticker,Name,SEDOL,Market Price ($),Shares Held,Market Value ($)
0,12.79,AAPL,APPLE INC,2046251,165.12,"5,059,138.00","835,364,866.56"
1,10.59,MSFT,MICROSOFT CORP,2588173,298.79,"2,315,126.00","691,736,497.54"
2,7.37,AMZN,AMAZON.COM INC,2000019,"3,071.26","156,642.00","481,088,308.92"
3,4.14,TSLA,TESLA INC,B616C79,870.43,"310,536.00","270,299,850.48"
4,4.05,NVDA,NVIDIA CORP,2379504,243.85,"1,085,828.00","264,779,157.80"
...,...,...,...,...,...,...,...
98,0.18,SWKS,SKYWORKS SOLUTIONS INC,2961053,138.17,"85,943.00","11,874,744.31"
99,0.18,DOCU,DOCUSIGN INC,BFYT7B7,118.43,"98,271.00","11,638,234.53"
100,0.17,PDD,PINDUODUO INC-ADR,BYVW0F7,51.86,"207,277.00","10,749,385.22"
101,0.14,SPLK,SPLUNK INC,B424494,118.10,"78,121.00","9,226,090.10"


In [4]:
qlyd = pd.read_csv('qyld_full-holdings_20220228.csv').dropna()

In [5]:
# Download the stock data
import os

folder = 'QLYD'
path = os.path.join(os.getcwd(), folder)

for ticker in qlyd.loc[:, 'Ticker']:
    try: 
        if not os.path.exists(os.path.join(path, ticker + '.csv')):
            DataLoader(ticker, dname='QLYD')
    except Exception as e:
        print(e)

# Analysis

In [6]:
def batching(data, period):
    batches = len(data) // period
    max_length = batches * period
    return np.array(data)[-max_length:].reshape(-1, period)

def regroup(data, period):
    opens = batching(data['Open'], period)[:, 0]
    high = batching(data['High'], period).max(1)
    low = batching(data['Low'], period).min(1)
    close = batching(data['Close'], period)[:, -1]
    volume = batching(data['Volume'], period).sum(1)
    date = pd.to_datetime(batching(data.index, period)[:, -1])
    return pd.DataFrame(np.stack([opens, high, low, close, volume], axis=1),
                        columns=['Open', 'High', 'Low', 'Close', 'Volume'],
                        index=date)

### Tables

In [7]:
def calculation(data):
    base = data['Close'].shift(-1) / data['Close'] - 1
    gain = data['High'].shift(-1) / data['Close'] - 1
    loss = data['Low'].shift(-1) / data['Close'] - 1
    df = pd.concat([base, gain, loss], axis=1).dropna()
    df.columns = ['Base', 'Gain', 'Loss']
    return df

In [8]:
period = 5
ticker = 'TSLA'
data = DataLoader(ticker).get_data('2016-01-01', '2017-12-31')
reshaped = regroup(data, period)
gains = calculation(reshaped)

In [9]:
gains.mean(0)

Base    0.005962
Gain    0.041854
Loss   -0.035989
dtype: float64